In [1]:
import os
import sys


class TradingBotException(Exception):
    def __init__(self, error_message, error_details: sys):
        self.error_message = error_message
        _, _, exc_tb = error_details.exc_info()
        
        self.lineno = exc_tb.tb_lineno
        self.file_name = exc_tb.tb_frame.f_code.co_filename 
    
    def __str__(self):
        return "Error occured in python script name [{0}] line number [{1}] error message [{2}]".format(
            self.file_name, self.lineno, str(self.error_message)
        ) 
    

In [2]:
    
if __name__ == "__main__":
    try:
        a = 1/0
        print("This will not be printed") 
    except Exception as e:
        raise TradingBotException(e, sys)

TradingBotException: Error occured in python script name [C:\Users\sowmy\AppData\Local\Temp\ipykernel_4972\2222399519.py] line number [3] error message [division by zero]

In [14]:
import logging
import os
from datetime import datetime

# Create 'logs/' directory if it doesn't exist
LOG_DIR = os.path.join(os.getcwd(), "logs")
os.makedirs(LOG_DIR, exist_ok=True)

# Create unique log file name
LOG_FILE = f"{datetime.now().strftime('%m_%d_%Y_%H_%M_%S')}.log"
LOG_FILE_PATH = os.path.join(LOG_DIR, LOG_FILE)

# Logging configuration
logging.basicConfig(
    filename=LOG_FILE_PATH,
    format="[ %(asctime)s ] %(lineno)d %(name)s - %(levelname)s - %(message)s",
    level=logging.INFO,
)

logger = logging.getLogger("my_agentic_app")

logging.info("hi")

[ 2025-05-13 15:21:49,867 ] 22 root - INFO - hi
